In [135]:
# import libraries
import numpy as np
import pandas as pd
from pathlib import Path

In [136]:
# read the dataframe
data_path = Path.cwd().parent / 'data' / 'paris_solcast_raw.csv'
df = pd.read_csv(data_path)

In [137]:
df.sample(5)

,PeriodEnd,PeriodStart,Period,AirTemp,CloudOpacity,Dhi,Dni,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m
9429,2020-01-27T23:00:00Z,2020-01-27T22:00:00Z,PT60M,8.5,12.7,0,0,0,12.0,75.8,990.8,230,6.9
8580,2019-12-23T14:00:00Z,2019-12-23T13:00:00Z,PT60M,10.5,48.7,103,0,103,12.0,63.8,1005.4,264,5.2
20596,2021-05-07T06:00:00Z,2021-05-07T05:00:00Z,PT60M,4.4,0.0,43,463,125,7.6,87.1,1005.0,279,2.1
20265,2021-04-23T11:00:00Z,2021-04-23T10:00:00Z,PT60M,14.8,0.0,92,947,813,6.0,37.9,1016.1,72,4.6
18145,2021-01-25T03:00:00Z,2021-01-25T02:00:00Z,PT60M,2.2,65.8,0,0,0,8.4,81.6,989.2,273,5.3


### Changing the names and types of the columns

In [138]:
old_names = ['PeriodEnd', 'PeriodStart', 'AirTemp', 'Dhi', 'Dni', 'Ghi','PrecipitableWater', 'RelativeHumidity',
             'SurfacePressure', 'WindDirection10m', 'WindSpeed10m'] 
new_names = ['PeriodEnd','PeriodStart','Temperature', 'DHI', 'DNI', 'Radiation', 'Precipitation', 'Humidity', 
             'Pressure', 'WindDirection', 'WindSpeed']
df.rename(columns = dict(zip(old_names, new_names)), inplace = True)
df = df.drop(columns=['Period'])

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26085 entries, 0 to 26084
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PeriodEnd      26085 non-null  object 
 1   PeriodStart    26085 non-null  object 
 2   Temperature    26085 non-null  float64
 3   CloudOpacity   26085 non-null  float64
 4   DHI            26085 non-null  int64  
 5   DNI            26085 non-null  int64  
 6   Radiation      26085 non-null  int64  
 7   Precipitation  26085 non-null  float64
 8   Humidity       26085 non-null  float64
 9   Pressure       26085 non-null  float64
 10  WindDirection  26085 non-null  int64  
 11  WindSpeed      26085 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 2.4+ MB


In [140]:
 # interpret columns as appropriate data types to ensure compatibility
for col in ['Radiation', 'Temperature', 'Pressure', 'Humidity', 'WindDirection', 'WindSpeed']:
    df[col] = df[col].astype(float)

for col in ['PeriodStart', 'PeriodEnd']:
    df[col] = pd.to_datetime(df[col]).dt.to_period('T').dt.to_timestamp()

C:\Users\maxlu\Anaconda3\envs\myenv\lib\site-packages\pandas\core\arrays\datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [141]:
df.head(5)

,PeriodEnd,PeriodStart,Temperature,CloudOpacity,DHI,DNI,Radiation,Precipitation,Humidity,Pressure,WindDirection,WindSpeed
0,2018-12-31 02:00:00,2018-12-31 01:00:00,7.9,0.0,0,0,0.0,17.7,97.9,1021.7,281.0,2.6
1,2018-12-31 03:00:00,2018-12-31 02:00:00,7.7,0.0,0,0,0.0,17.1,98.2,1021.6,284.0,2.7
2,2018-12-31 04:00:00,2018-12-31 03:00:00,7.7,0.0,0,0,0.0,16.7,98.4,1021.4,288.0,2.6
3,2018-12-31 05:00:00,2018-12-31 04:00:00,7.6,0.0,0,0,0.0,16.6,98.3,1021.2,296.0,2.5
4,2018-12-31 06:00:00,2018-12-31 05:00:00,7.6,9.2,0,0,0.0,16.5,98.3,1021.1,303.0,2.5


### Aplying correct time zone and winter/summer time

In [142]:
df['PeriodStart'] = df['PeriodStart'].dt.tz_localize('UTC').dt.tz_convert('Europe/Paris')
df['PeriodEnd'] = df['PeriodEnd'].dt.tz_localize('UTC').dt.tz_convert('Europe/Paris')

### Create sunrise and sunset feature

In [143]:
# Create new features(year, month, day) from start and end time of data collection  
df['Year'] = pd.to_datetime(df['PeriodEnd']).dt.year
df['Month'] = pd.to_datetime(df['PeriodEnd']).dt.month

df['HourPS'] = pd.to_datetime(df['PeriodStart']).dt.hour
df['HourPE'] = pd.to_datetime(df['PeriodEnd']).dt.hour

In [144]:
df.loc[df['Radiation'] == 0]['HourPS'].unique()

array([ 2,  3,  4,  5,  6,  7, 17, 18, 19, 20, 21, 22, 23,  0,  1,  8],
      dtype=int64)

In [145]:
df.loc[df['Radiation'] == 0]['HourPE'].unique()

array([ 3,  4,  5,  6,  7,  8, 18, 19, 20, 21, 22, 23,  0,  1,  2,  9],
      dtype=int64)

In [146]:
# create a new feature for sunrise using the function below; fill time for sunrise
def time_to_sunrise(row):
    """
    A function for identifying time of sunrise.
    """
    value = 0
    if row['HourPS'] == 8 and row['HourPE'] == 9 and row['Radiation'] > 0:
        value= '09:00:00'
    if row['HourPS'] == 7 and row['HourPE'] == 8 and row['Radiation'] > 0:
        value= '08:00:00'
    elif row['HourPS'] == 6 and row['HourPE'] == 7 and row['Radiation'] > 0:
        value= '07:00:00'
    elif row['HourPS'] == 5 and row['HourPE'] == 6 and row['Radiation'] > 0:
        value= '06:00:00'
    elif row['HourPS'] == 4 and row['HourPE'] == 5 and row['Radiation'] > 0:
        value= '05:00:00'
    elif row['HourPS'] == 3 and row['HourPE'] == 4 and row['Radiation'] > 0:
        value= '04:00:00'
    return value

# create a new feature for sunset using the function below; fill time for sunset
def time_to_sunset(row):
    """
    A function for identifying time of sunset.
    """
    value= 0
    if row['HourPS'] == 15 and row['HourPE'] == 16 and row['Radiation']> 0:
        value= '16:00:00'
    elif row['HourPS'] == 16 and row['HourPE'] == 17 and row['Radiation']> 0:
        value= '17:00:00'
    elif row['HourPS'] == 17 and row['HourPE']== 18 and row['Radiation']> 0:
        value= '18:00:00'
    elif row['HourPS'] == 18 and row['HourPE']== 19 and row['Radiation']> 0:
        value= '19:00:00'
    elif row['HourPS'] == 19 and row['HourPE']== 20 and row['Radiation']> 0:
        value= '20:00:00'
    elif row['HourPS'] == 20 and row['HourPE']== 21 and row['Radiation']> 0:
        value= '21:00:00'
    elif row['HourPS'] == 21 and row['HourPE']== 22 and row['Radiation']> 0:
        value= '22:00:00'
    elif row['HourPS'] == 22 and row['HourPE']== 23 and row['Radiation']> 0:
        value= '23:00:00'
    return value

In [147]:
#Create a new column using the function above
df['Sunrise'] = df.apply(time_to_sunrise, axis=1)

#Create a new column using the function above
df['Sunset'] = df.apply(time_to_sunset, axis=1)

In [148]:
df.Sunrise.unique(), df.Sunset.unique()

(array([0, '09:00:00', '08:00:00', '07:00:00', '06:00:00'], dtype=object),
 array([0, '16:00:00', '17:00:00', '18:00:00', '19:00:00', '20:00:00',
        '21:00:00', '22:00:00'], dtype=object))

### Removing night time data

In [149]:
# changing integer values to datetime format
df = df.loc[~((df['Radiation'] == 0) & (df['DNI'] == 0) & (df['DHI'] == 0)),:]

### Grouping by day

In [150]:
df['date'] = pd.to_datetime(df['PeriodEnd']).dt.date

for col in ['Sunrise', 'Sunset']:
    df[col] = df[col].astype(str)
    
agg_dict = {
    'Temperature':'mean',
    'CloudOpacity':'mean',
    'DHI':'mean',
    'DNI':'mean',
    'Radiation':'mean',
    'Precipitation':'mean',
    'Humidity':'mean',
    'Pressure':'mean',
    'WindDirection':'mean',
    'WindSpeed':'mean',
    'Sunrise':'max',
    'Sunset':'max',
}

df_grouped = df.groupby(['date']).agg(agg_dict)
df_grouped.reset_index(inplace=True)
df_grouped

,date,Temperature,CloudOpacity,DHI,DNI,Radiation,Precipitation,Humidity,Pressure,WindDirection,WindSpeed,Sunrise,Sunset
0,2018-12-31,7.888889,65.944444,35.777778,0.000000,35.777778,13.555556,95.400000,1021.477778,329.111111,1.855556,09:00:00,17:00:00
1,2019-01-01,6.922222,68.077778,36.111111,0.000000,36.111111,9.877778,90.033333,1019.522222,306.777778,3.544444,09:00:00,17:00:00
2,2019-01-02,4.266667,40.544444,70.888889,13.555556,72.000000,6.833333,74.255556,1024.744444,10.777778,4.577778,09:00:00,17:00:00
3,2019-01-03,2.800000,35.677778,83.111111,18.111111,84.666667,7.666667,83.722222,1025.077778,21.333333,3.222222,09:00:00,17:00:00
4,2019-01-04,2.033333,52.322222,67.333333,21.777778,71.555556,8.544444,86.611111,1024.966667,53.555556,3.088889,09:00:00,17:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082,2021-12-17,7.344444,47.888889,46.444444,100.000000,62.666667,11.244444,79.222222,1025.877778,56.000000,3.688889,09:00:00,17:00:00
1083,2021-12-18,6.588889,0.000000,31.888889,523.333333,154.111111,9.111111,72.755556,1024.577778,47.222222,3.200000,09:00:00,17:00:00
1084,2021-12-19,4.688889,35.577778,92.555556,0.000000,92.555556,7.522222,81.922222,1020.800000,43.555556,3.122222,09:00:00,17:00:00
1085,2021-12-20,6.166667,68.200000,33.555556,0.000000,33.555556,8.044444,69.700000,1015.777778,66.111111,3.522222,09:00:00,17:00:00


In [151]:
df_grouped.shape

(1087, 13)

In [152]:
output_path = Path.cwd().parent / 'data' / 'paris_solcast_cleaned.csv'
df_grouped.to_csv(output_path)